In [3]:
%load_ext autoreload
%autoreload 2

In [23]:
import pandas as pd
import lightgbm as lgb

import mtg

In [24]:
past_games = mtg.PastGames()

In [26]:
past_games.cards

['Alessin_Adamant_Angel',
 'Bold_Battalion',
 'Dreadwing_Darkfire_Dragon',
 'Evil_Emperor_Eschatonus_Empyreal_Envoy_of_Entropic_End',
 'Gentle_Guard',
 'Horrible_Hooligan',
 'Kindly_Knight',
 'Lilac_Lotus',
 'Murderous_Minotaur',
 'Patchy_Pirate',
 'Sword_of_Shadows',
 'Virtuous_Vigilante']